# ライブラリのインポート

In [89]:
import pandas as pd  # 表データを扱うライブラリ
import numpy as np  # 数値計算用のライブラリ
import matplotlib.pyplot as plt # 可視化用のライブラリ
import seaborn as sns  #　綺麗に可視化することができるライブラリ
import warnings
warnings.filterwarnings('ignore')  # 警告を無視する
import time
import glob

### Jupyter Notebookで、pandasの表示行数や表示列数を変更する

In [96]:
#現在の最大表示行数の出力
print(pd.get_option("display.max_rows"))

#最大表示行数の指定（ここでは　１００行を指定）
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

100


## データの読み込み

In [97]:
# BTCファイルの読み込みと連結
temp_btc_files = sorted(glob.iglob('./data/BTC/BTC_JPY_ohlcv_*.pkl'))
df_list = []
for btc_file in temp_btc_files:
    temp_df = pd.read_pickle(btc_file)
    df_list.append(temp_df)

btc_df = pd.concat(df_list)

In [98]:
# FX_BTCファイルの読み込みと連結
temp_fxbtc_files = sorted(glob.iglob('./data/FX_BTC/FX_BTC_JPY_ohlcv_*.pkl'))
df_list = []
for fxbtc_file in temp_fxbtc_files:
    temp_df = pd.read_pickle(fxbtc_file)
    df_list.append(temp_df)

fxbtc_df = pd.concat(df_list)

### データの中身を確認

In [134]:
btc_df.columns

Index(['op', 'hi', 'lo', 'cl', 'volume', 'volume_buy_sum', 'volume_sell_sum',
       'volume_buy', 'volume_sell'],
      dtype='object')

In [100]:
fxbtc_df

,op,hi,lo,cl,volume,volume_buy_sum,volume_sell_sum,volume_buy,volume_sell
timestamp,,,,,,,,,
2018-01-01 00:01:00+00:00,1884160.0,1889890.0,1884160.0,1888999.0,12.453978,6147.622671,4781.211680,3.171679,9.282299
2018-01-01 00:02:00+00:00,1888999.0,1889892.0,1884161.0,1885787.0,24.037741,5969.723071,4817.574471,11.493328,12.544412
2018-01-01 00:03:00+00:00,1885787.0,1886074.0,1880740.0,1882043.0,30.480556,5813.856981,4713.962894,18.496474,11.984082
2018-01-01 00:04:00+00:00,1882043.0,1886247.0,1882000.0,1882961.0,20.839496,5867.336392,4726.835472,11.470742,9.368754
2018-01-01 00:05:00+00:00,1882961.0,1884983.0,1879690.0,1882581.0,47.991520,6056.577528,4757.127376,32.205018,15.786502
...,...,...,...,...,...,...,...,...,...
2022-05-31 23:56:00+00:00,4092000.0,4092069.0,4088644.0,4090150.0,5.831372,145.452123,328.692970,4.356221,1.475150
2022-05-31 23:57:00+00:00,4090396.0,4093663.0,4089059.0,4091339.0,2.400130,145.222513,323.218699,0.772351,1.627779
2022-05-31 23:58:00+00:00,4091265.0,4094036.0,4089019.0,4089705.0,5.704033,147.471628,323.598319,4.748328,0.955704


# バックテストを行う

## bitFlyerのSFD乖離を利用した戦略
- bitFlyerの現物と先物Bitcoinの価格乖離抑制の仕組みであるSFDを利用した戦略

### データの整理を行う
・1分足で [timestamp, 現物open, 現物close, 先物open, 先物close, SFDの値] のデータフレームに整形する

In [101]:
temp_df = pd.merge(btc_df, fxbtc_df, on='timestamp', suffixes=['_btc', '_fxbtc'], how='inner', sort=True)
df = temp_df.drop(columns=['hi_btc', 'lo_btc', 'volume_btc', 'volume_buy_btc', 'volume_buy_sum_btc', \
                           'volume_sell_btc', 'volume_sell_sum_btc', 'hi_fxbtc', 'lo_fxbtc', 'volume_fxbtc', 'volume_buy_fxbtc', \
                           'volume_buy_sum_fxbtc', 'volume_sell_fxbtc', 'volume_sell_sum_fxbtc'])

In [102]:
df

,op_btc,cl_btc,op_fxbtc,cl_fxbtc
timestamp,,,,
2018-01-01 00:01:00+00:00,1674700.0,1677680.0,1884160.0,1888999.0
2018-01-01 00:02:00+00:00,1677680.0,1675009.0,1888999.0,1885787.0
2018-01-01 00:03:00+00:00,1675009.0,1670279.0,1885787.0,1882043.0
2018-01-01 00:04:00+00:00,1670279.0,1669403.0,1882043.0,1882961.0
2018-01-01 00:05:00+00:00,1669403.0,1669040.0,1882961.0,1882581.0
...,...,...,...,...
2022-05-31 23:56:00+00:00,4092100.0,4090385.0,4092000.0,4090150.0
2022-05-31 23:57:00+00:00,4090000.0,4092225.0,4090396.0,4091339.0
2022-05-31 23:58:00+00:00,4091997.0,4090869.0,4091265.0,4089705.0


In [103]:
df['sfd'] = (df['cl_fxbtc'] - df['cl_btc']) / df['cl_fxbtc']

In [131]:
df

,op_btc,cl_btc,op_fxbtc,cl_fxbtc,sfd
timestamp,,,,,
2018-01-01 00:01:00+00:00,1674700.0,1677680.0,1884160.0,1888999.0,0.111868
2018-01-01 00:02:00+00:00,1677680.0,1675009.0,1888999.0,1885787.0,0.111772
2018-01-01 00:03:00+00:00,1675009.0,1670279.0,1885787.0,1882043.0,0.112518
2018-01-01 00:04:00+00:00,1670279.0,1669403.0,1882043.0,1882961.0,0.113416
2018-01-01 00:05:00+00:00,1669403.0,1669040.0,1882961.0,1882581.0,0.113430
...,...,...,...,...,...
2022-05-31 23:56:00+00:00,4092100.0,4090385.0,4092000.0,4090150.0,-0.000057
2022-05-31 23:57:00+00:00,4090000.0,4092225.0,4090396.0,4091339.0,-0.000217
2022-05-31 23:58:00+00:00,4091997.0,4090869.0,4091265.0,4089705.0,-0.000285


### SFDの値をプロットする

In [105]:
plt.plot(df['sfd'])
plt.axhline(y=0.048)
plt.axhline(y=-0.048)
plt.show()

# バックテストを行う

- ショートに関するentry/exit条件
  - ショートポジションのエントリー条件：0.048 < SFD
  - ショートポジションの手仕舞い条件：((0.04 > 0.048取得したSFDの現在の値) and ショートポジションを持っている)
- ロングに関するentry/exit条件
  - ロングポジションのエントリー条件：-0.048 > SFD
  - ロングポジションの手仕舞い条件：((0.0 < -0.048取得したSFDの現在の値) and ロングポジションを持っている)

In [106]:
# パラメータ
position = { "qty": 0, "avgEntry":0}
exec_history = []

In [107]:
def _market_order(size, next_open, timestamp):
    """
    do_backtest内でmarketオーダーを行う関数
    next_open : float
        次回のopenの価格（ここで成り行き注文が約定する
    size : float
        orderのサイズ　+ でロング　-でショート
    timestamp : Timestamp or Unixtime
        時刻
    """
    order = {
            "timestamp": timestamp,
            "size": size,
            "price": next_open
        }
    exec_history.append(order)  # itterableなオブジェクトは引数で与えなくても変更できる do_backtest内で使用する
    _change_position(size, next_open)

In [84]:
def _change_position(size, next_open):
    """
    オーダーをポジションに加える
    size : float
        orderのサイズ　+ でロング　-でショート
    """
    qty = position["qty"]
    avg = position["avgEntry"]
    new_qty = qty + size
    # 計算が必要になる
    if new_qty == 0:
        new_avg = 0
    else:
        # 同方向のポジション
        if qty * size >= 0:
            new_avg = (qty * avg + size * next_open) / (qty + size)
        # 違う方向のポジション
        else:
            if qty >= 0:
                # 元のポジションの方が大きい
                if new_qty > 0:
                    new_avg = avg
                # 今のポジションの方が大きい
                else:
                    new_avg = next_open
            # 元のポジションがマイナス
            else:
                if new_qty > 0:
                    new_avg = next_open
                # 今のポジションの方が大きい
                else:
                    new_avg = avg
    position["qty"] = new_qty
    position["avgEntry"] = new_avg

### バックテストを実行する関数

In [143]:
def do_backtest(df, order_lot): # 変数が必要であれば引数を追加する
    print(" ----   Do Backtest   ----")
    start = time.time()
    
    #  カラム名がtimestampでもtimeでも対応可能
#    if "datetime" in df.columns:
#        time_record = df.index
#    elif "timestamp" in df.columns :
#        time_record = df["timestamp"].index
#    elif "time" in df.columns :
#       time_record = df["time"].index
#   else:
#        print('else')
    timestamp = df.index
    
    # -------------------------- 必要なカラムの情報を入れてください ----------------------------
    op_btc = df["op_btc"].values
    cl_btc = df["cl_btc"].values
    op_fxbtc = df["op_fxbtc"].values
    cl_fxbtc = df["cl_fxbtc"].values
    sfd = df["sfd"].values
    
    # 各閾値
    l_th = -0.048 # ロングポジションのSFD
    s_th = 0.048 # ショートポジションのSFD
    ltp_th = 0.04 # ロングポジション手仕舞いのSFD
    stp_th = 0.0 # ショートポジション手仕舞いのSFD
    
    for i in range(len(df)-1):
        # for文はzipで回しても良い
        o_btc = op_btc[i]
        c_btc = cl_btc[i]
        o_fx = op_fxbtc[i]
        c_fx = cl_fxbtc[i]
        next_op_fxbtc = op_fxbtc[i+1] # 次のopen価格
        sfd_value = sfd[i]
        t = timestamp[i]
        qty = position["qty"]  # ポジションサイズ
        avgEntry = position["avgEntry"]  # 平均エントリー価格

        # -------------------------- ここから戦略 ----------------------------
        # longCondition = l_th > sfd              // -0.048 > sfdの値
        # shortCondition = s_th < sfd          // 0.048 < sfdの値
        # ロングポジションの手仕舞い条件：((0.0 < -0.048取得したSFDの現在の値) and ロングポジションを持っている)
        # ショートポジションの手仕舞い条件：((0.04 > 0.048取得したSFDの現在の値) and ショートポジションを持っている)

        # ポジションがない場合
        if qty == 0:
            if l_th > sfd_value:
                # ロングポジションを持つ
                _market_order(order_lot, next_op_fxbtc, t)
            elif s_th < sfd_value:
                #　ショートポジションを持つ
                _market_order(-order_lot, next_op_fxbtc, t)
        elif qty > 0 and ltp_th <= sfd_value:
            # ロングポジションの手仕舞い
            _market_order(-order_lot * qty, next_op_fxbtc, t)
        elif qty < 0 and stp_th >= sfd_value:
            # ショートポジションの手仕舞い
            _market_order(order_lot * qty, next_op_fxbtc, t)
       
    print(" ----   finish backtest   ----")
    elapsed_time = time.time() - start
    print(f" ----   elapsed time: {elapsed_time}   ---- ")
    return pd.DataFrame(exec_history, columns=["timestamp", "size", "price"])

### バックテストの実行

In [145]:
exec_history_df  = exec_history_df = do_backtest(df, 0.001)

 ----   Do Backtest   ----
 ----   finish backtest   ----
 ----   elapsed time: 18.049171924591064   ---- 


In [146]:
exec_history

[{'timestamp': Timestamp('2018-01-01 00:01:00+0000', tz='UTC'),
  'size': -0.001,
  'price': 1888999.0},
 {'timestamp': Timestamp('2018-01-16 09:28:00+0000', tz='UTC'),
  'size': -1e-06,
  'price': 1515000.0},
 {'timestamp': Timestamp('2018-01-16 09:29:00+0000', tz='UTC'),
  'size': -1.001e-06,
  'price': nan},
 {'timestamp': Timestamp('2018-01-16 09:35:00+0000', tz='UTC'),
  'size': -1.002001e-06,
  'price': nan},
 {'timestamp': Timestamp('2018-01-16 09:42:00+0000', tz='UTC'),
  'size': -1.003003001e-06,
  'price': 1320700.0},
 {'timestamp': Timestamp('2018-01-16 09:43:00+0000', tz='UTC'),
  'size': -1.004006004001e-06,
  'price': 1316168.0},
 {'timestamp': Timestamp('2018-01-16 09:44:00+0000', tz='UTC'),
  'size': -1.0050100100050009e-06,
  'price': 1316158.0},
 {'timestamp': Timestamp('2018-01-16 09:45:00+0000', tz='UTC'),
  'size': -1.006015020015006e-06,
  'price': 1293901.0},
 {'timestamp': Timestamp('2018-01-16 09:46:00+0000', tz='UTC'),
  'size': -1.007021035035021e-06,
  'pric

In [147]:
exec_history_df

,timestamp,size,price
0,2018-01-01 00:01:00+00:00,-1.000000e-03,1888999.0
1,2018-01-16 09:28:00+00:00,-1.000000e-06,1515000.0
2,2018-01-16 09:29:00+00:00,-1.001000e-06,NaN
3,2018-01-16 09:35:00+00:00,-1.002001e-06,NaN
4,2018-01-16 09:42:00+00:00,-1.003003e-06,1320700.0
...,...,...,...
407114,2022-05-31 23:54:00+00:00,-5.231001e+170,4093371.0
407115,2022-05-31 23:55:00+00:00,-5.236232e+170,4092000.0
407116,2022-05-31 23:56:00+00:00,-5.241469e+170,4090396.0
407117,2022-05-31 23:57:00+00:00,-5.246710e+170,4091265.0
